In [1]:
import numpy as np
from scipy.interpolate import griddata

In [2]:
def reading_field(head,name,n_package,file_name,i,j):#Reading the raw vector data
    file = head +name +'/'+ n_package + '/' + file_name
    raw_data = []
    number = 23 + (i * j)
    with open(file) as f:
        for line in f:
            line = line.strip('\n')
            raw_data.append(line)
    data_str = (raw_data[23:number])
    data  = [float(i) for i in data_str]
    data = np.array(data)
    return data

def reading_U(head,name,n_package,file_name,i,j): # Reading second-rank tensor in OpenFoam field
    file = head +name +'/'+ n_package + '/' + file_name
    raw_data = []
    number = (i * j)
    with open(file) as f:
        for line in f:
            line = line.strip('\n ()')
            raw_data.append(line)
    data_str = raw_data[23:(23+number)]
    tensors = np.zeros((number,3))
    for i in range(number):
        element = np.array([float(i) for i in data_str[i].split()])
        tensors[i] = element
    U = tensors[:,0]
    V = tensors[:,1]
    return U,V

def Epsilon(omega,k):
    ep = []
    for i in range(len(omega)):
        ep.append(0.09*omega[i]*k[i])
    #ep = np.array(ep)
    return ep
def Delta(U,U_):
    D = max(U_) - min(U_)
    delta = []
    for i in range(len(U)):
        delta.append(abs(U[i]-U_[i])/D)
    delta = np.array(delta)
    return delta

In [13]:
case = ['h5','h8','h10']
file = ['10000','10000','14000']
head ='/home/haoyan/Desktop/ML-CFD/CASE/PHill/Grid-iretation-independency/'
i = [70,93,140]
j = [65,83,120]

In [15]:
x2 = reading_field(head,case[0],file[0],'Cx',i[0],j[0])
y2 = reading_field(head,case[0],file[0],'Cy',i[0],j[0])
x1 = reading_field(head,case[1],file[1],'Cx',i[1],j[1])
y1 = reading_field(head,case[1],file[1],'Cy',i[1],j[1])
x0 = reading_field(head,case[2],file[2],'Cx',i[2],j[2])
y0 = reading_field(head,case[2],file[2],'Cy',i[2],j[2])

In [16]:
U2,V2 = reading_U(head,case[0],file[0],'U',i[0],j[0])
U1,V1 = reading_U(head,case[1],file[1],'U',i[1],j[1])
U0,V0 = reading_U(head,case[2],file[2],'U',i[2],j[2])
k2 = reading_field(head,case[0],file[0],'k',i[0],j[0])
k1 = reading_field(head,case[1],file[1],'k',i[1],j[1])
k0 = reading_field(head,case[2],file[2],'k',i[2],j[2])
omega2 = reading_field(head,case[0],file[0],'omega',i[0],j[0])
omega1 = reading_field(head,case[1],file[1],'omega',i[1],j[1])
omega0 = reading_field(head,case[2],file[2],'omega',i[2],j[2])

In [22]:
U0_inter = griddata((x0,y0),U0,(x1,y1),method = 'cubic')
V0_inter = griddata((x0,y0),V0,(x1,y1),method = 'cubic')
k0_inter = griddata((x0,y0),k0,(x1,y1),method = 'cubic')
omega0_inter = griddata((x0,y0),omega0,(x1,y1),method = 'cubic')
U0_inter1 = griddata((x0,y0),U0,(x2,y2),method = 'cubic')
V0_inter1 = griddata((x0,y0),V0,(x2,y2),method = 'cubic')
k0_inter1 = griddata((x0,y0),k0,(x2,y2),method = 'cubic')
omega0_inter1 = griddata((x0,y0),omega0,(x2,y2),method = 'cubic')
U1_inter = griddata((x1,y1),U1,(x2,y2),method = 'cubic')
V1_inter = griddata((x1,y1),V1,(x2,y2),method = 'cubic')
k1_inter = griddata((x1,y1),k1,(x2,y2),method = 'cubic')
omega1_inter = griddata((x1,y1),omega1,(x2,y2),method = 'cubic')

In [18]:
U1_U2 = max(Delta(U2,U1_inter))
print(U1_U2)
V1_V2 = max(Delta(V2,V1_inter))
print(V1_V2)
k1_k2 = max(Delta(k2,k1_inter))
print(k1_k2)
o1_o2 =  max(Delta(omega2,omega1_inter))
print(o1_o2)

0.7643848221813909
0.26487218075627106
0.10941677636554686
0.5156557845854046


In [23]:
U0_U1 = max(Delta(U1,U0_inter))
print(U0_U1)
V0_V1 = max(Delta(V1,V0_inter))
print(V0_V1)
k0_k1 = max(Delta(k1,k0_inter))
print(k0_k1)
o0_o1 =  max(Delta(omega1,omega0_inter))
print(o0_o1)

0.5660281140538794
0.16585411563897892
0.16418003652800392
0.5051442967240676


In [25]:
U0_U2 = max(Delta(U2,U0_inter1))
print(U0_U2)
V0_V2 = max(Delta(V2,V0_inter1))
print(V0_V2)
k0_k2 = max(Delta(k2,k0_inter1))
print(k0_k2)
o0_o2 =  max(Delta(omega2,omega0_inter1))
print(o0_o2)

0.23923118546564656
0.24531443764017627
0.10241926035757029
0.5242651275682918
